![](./images/header.png)

# 1. Introduction

In this notebook, we'll start from a time series of Sentinel-2 data, as well as a in-situ database of crop types in Malawi to train a crop type mapping algorithm.

By using the prior knowledge obtained from the previous exercise, we will preprocess the data, extract the data at our in-situ data locations, perform feature extraction and train a Random Forest Classifier.

First, let's explore the in-situ data.

# 2. Explore the in-situ data

By loading our in-situ dataframe, we can assess the variety of the dataset, as well as the location of the fields.


To start, load the in-situ dataset and visualize the amount and variety of training samples.

In [ ]:
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
# Load the dataset
inpath = Path('/data/users/Public/couchard/Malawi_normalized_data_2023.gpkg')
croptype_df = <YOUR CODE HERE>

# Visualize the first few rows
croptype_df.head()

In [ ]:
# Here we count the amount of samples for each different croptype
value_counts = croptype_df['croptype'].value_counts()

# Let's make a bar plot to show the amount of samples for each croptype
<YOUR CODE HERE>

We can see that there is healthy amount of samples for Soy beans, Three crops, Maize, Oilseed and Tobacco. The other croptypes are too scarse in amount of points. When there are not enough data samples, it can be difficult to train a classifier.

Select only the first 4 most dominant crop types, and rename the other croptypes to the "Other" class. Do not consider the class, "Mixture of threes or more"

In [ ]:
croptype_df = <YOUR CODE HERE>
croptype_df.head()

Now that you we have our filterered dataset, evaluate the bounding box of the points.

Using GeoPandas plotting functionalities, create an bounding box enveloping the entirety of our training dataset. To give it a comparison, plot it against the country of Malawi.

Documentation for those functionalities is available here:
https://geopandas.org/en/stable/docs/user_guide/mapping.html

In [ ]:
# Loads a dataset containing all the countries borders, and search for Malawi.
# Plot the malawi border and adds the bounding box envelopping all our training data points on the plot.
<YOUR CODE HERE>

# 3. Load and preprocess the training data

Using the same Sentinel-2 pre-processing piepeline used in the previous exercise, load the data and perform the same pre-processing operations.

* Compute the cloud mask from the 'SCENECLASSIFICATION' layer.
* Load the 20-m array, mask the clouds, perform 10-daily temporal compositing and linear interpolation.
* Using the same cloud mask, do the same cloud masking, temporal compositing and interpolation with the 10-m array.

No need to upsample the 20-m array.

In [ ]:
# Import the necessary python libraries...
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

# import custom python functions all defined in separate .py files
# custom functions
from mask import mask_ts
from composite import composite_ts
from interpolate import interpolate_ts

In [ ]:
infile_20m = '/data/users/Public/couchard/S2_L2A_Malawi_20m.nc'
infile_10m = '/data/users/Public/couchard/S2_L2A_Malawi_10m.nc'

ds_20 = xr.open_dataset(infile_20m)

ts_20_fin = <YOUR CODE HERE>

In [ ]:
ds_10 = xr.open_dataset(infile_10m)

ts_10_fin = <YOUR CODE HERE>

Compute the NDVI value, as done in the previous exercise.

Then, using the <b>xr.concat(...)</b> function, assemble the two preprocessed arrays and the NDVI into a single array.

Docs: https://docs.xarray.dev/en/stable/user-guide/combining.html#concatenate

In [ ]:
ts_fin = xr.concat([ts_10_fin, ts_20_fin], dim='variable')

ndvi = <YOUR CODE HERE>

# Concatenate the ndvi with the other bands, forming an entire array
ts_fin = <YOUR CODE>

In [ ]:
# You can free memory from your compute by removing all the intermediate arrays used for cloud masking, compositing, linear interpolation...
del <YOUR INTERMEDIARY ARRAYS>

# 4. Train a crop type classifier using Machine Learning.

In order to create a crop type map in malawi, we first need to train a Random Forest Classifier with the in-situ data we have available.

In this section, you will have to:

* Sample points within the training data we have at our disposition
* For every point, combine the variable and temporal dimension in to a single dimension using quentiles/percentiles.
* Train de model, evaluate its performance and perform full-tile inference.

### Sample random points from the GeoPandas dataset

From the GeoPandas dataset of croptypes constructed previously in this exercise, sample random points using the `geopandas.GeoSeries.sample_points` function.

Then, sample those points from the array that we just assembled

Docs: https://geopandas.org/en/stable/docs/user_guide/sampling.html

In [ ]:
POINTS_PER_FIELD = 10

sampled_points = <YOUR CODE HERE>

# Prepare the point labels so 
point_labels = croptype_df.croptype.repeat(POINTS_PER_FIELD)

# Visualize the sampled points against the croptype dataset
<YOUR CODE HERE>

Using the `extract_points` function, extract the sampled points from the `ts_fin` sensor array.

In [ ]:
from extract import extract_points

training_data = <YOUR CODE HERE>
training_data

### Compute additional features using percentiles

One of the most interesting aspect of Remote Sensing when using Machine Learning is the temporal dimension. Different objects visible on the surface of earth will have a different evolution of with time. The most notable examples are usually in vegetation, were seasonal properties exists depending on the region and the species. Looking at the temporal dimension can therefore improve classification tasks. Recent research push Deep Learning techniques to analyse those temporal characteristics and use the most of it.

Unfortunately, analyzing the temporal dimension obviously implies the use of another dimension, increasing tremendously the computational power requirements. This does not help the realization of large Remote Sensing projects, which already suffer from the very large quantities of data to process.

There is a large variety of techniques that can be used to mitigate the issue by combining the `variable` and the `time` dimensions into one. Here, we will use percentiles/quantiles computed on the `time` dimension. 

Using the `band_percentile` function, extract the following percentiles:

* For the Red, Green, Blue and NIR bands, we compute the 10th, 50th and 90th percentiles.
* For the NDVI, we compute the 10th, 25th, 50th, 75th and 90th percentiles.
* For the other bands, we only keep the 50th percentile (median)

In [ ]:
from features import band_percentile

band_percentiles = {
    'B02': [0.1, 0.5, 0.9],
    'B03': [0.1, 0.5, 0.9],
    'B04': [0.1, 0.5, 0.9],
    'B08': [0.1, 0.5, 0.9],
    'NDVI': [0.1, 0.25, 0.5, 0.75, 0.9],
    'B05': [0.5],
    'B06': [0.5],
    'B07': [0.5],
    'B11': [0.5],
    'B12': [0.5]
}

training_data_df = band_percentile(training_data, band_percentiles).to_dataframe(
        name='features', dim_order=['sample', 'features']
).unstack()

training_data_df

### Balance the dataset ⚖️

Because the "Other" class is very dominant in our dataset, we will use a small feature from the `imbalance-learn` package, allowing to better equilibrate the dataset and have the same number of class for each croptype.

Having an equilibrated dataset is important as it prevents the model to overfit on a certain class.

Initialize a `RandomUnderSampler` object and perform resampling on the `train_data_df` and `point_labels` datasets.  

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

X, y = <YOUR CODE HERE>

### AI Time 💃 Let's train a Random Forest Classifier

Using the sklearn framework, let's train a random forest classifier. Before fitting the data within our model, we still need to do a final step: separating the data in two sets: the <b>training</b> data and the <b>validation</b> data. In order to prove that the model generalizes well on new samples, we want to test the performance of our model on different data than the one that was used to train it.

This split can be easily done from our DataFrame using the `sklearn.model_selection.train_test_split` function.

Docs: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Once split the training dataset split, train the model with it

Docs: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


# Let's perform dataset split
X_train, X_test, y_train, y_test = <YOUR CODE HERE>

# Initialize the model, you can use the default model parameters defined in Sklearn, as they are usually very good
model = <YOUR CODE HERE>

# We train our model
<YOUR CODE HERE>

### Evoluate model performance

To evaluate the model performance, we perform prediction on the validation data that we previously split from. Then, we can use the `sklearn.metrics.accuracy_score` and `sklearn.metrics.confusion_matrix` functions.

The accuracy score denotates how many validation examples were correctly labelled by the model, while the confusion matrix shows where the prediction errors are made by comparing in-situ crop types with crop types predicted by the model.

Docs: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

# First, we predict the result the model on the validation data
y_pred = <YOUR CODE HERE>

# Compute the accuracy score, the confusion matrix and the plot the confusion matrix
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(accuracy)

plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)

# Set labels, title, and ticks
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.xticks(np.arange(len(cm)) + 0.5, labels=model.classes_, rotation=30)  # Replace labels with your class names
plt.yticks(np.arange(len(cm)) + 0.5, labels=model.classes_, rotation=30)  # Replace labels with your class names

### Perform inference for crop type mapping 🗺️

Now that our model is trained, let's perform inference on the entirety of our downloaded data.

First, compute the same quantiles as done for the training data, but this time on the entire data cube instead of the samples datacube. Then combine those quantile cubes into a single one using the xr.concat method.

In [ ]:
ts_inference = <YOUR CODE HERE>

inference_df = ts_inference.to_dataframe(
    name='features', dim_order=['x', 'y', 'features']
).unstack()

inference_df

As done previously, transform the entire datacube into a single dataset, then perform model inference using the `model.predict` method.

In [ ]:
prediction = <YOUR CODE HERE>
prediction

### Map visualization 🔎

Let's reconstruct the map from the given prediction. Let's map every class to a tuple giving a color.

In [ ]:
# Blue
OTHER_COLOR = (48, 110, 209)

# Yellow
MAIZE = (204, 164, 55)

# Purple
SOY_BEANS = (163, 5, 158)

# Olive color
OILSEEDS_CROPS = (219, 245, 91)

# Brown
TOBACCO = (107, 55, 55)

color_map = {
    'tobacco': TOBACCO,
    'oilseed_crops': OILSEEDS_CROPS,
    'soy_soybeans': SOY_BEANS,
    'Other': OTHER_COLOR,
    'maize': MAIZE
}

def map_class_to_color(class_value: str):
    # Returns the color of the given class, and black color if unknown
    return color_map.get(class_value, (0, 0, 0))

vectorized_map = np.vectorize(map_class_to_color)

rgb_array = np.array(vectorized_map(prediction)).squeeze().reshape(
    3, ts_inference.shape[1], ts_inference.shape[0]
)

rgb_array = np.moveaxis(rgb_array, 0, -1)

prediction_array = xr.DataArray(
    rgb_array,
    dims=['x', 'y', 'channels'],
    coords={
        'x': ts_inference.coords['x'],
        'y': ts_inference.coords['y'],
        'channels': ['R', 'G', 'B']
    }
)

prediction_array.shape

In [ ]:
fig, axis = plt.subplots(ncols=2, nrows=1, figsize=(12, 6))

rgb = (ts_inference.sel(features=['B04-0.5', 'B03-0.5', 'B02-0.5']) / 1e4) ** .4

axis[0].imshow(rgb)
axis[1].imshow(rgb_array)

In [ ]:
import hvplot.xarray

sensor_layer = rgb.hvplot.rgb(x='x', y='y', bands='features', aspect=1, width=500)
prediction_layer = prediction_array.transpose('channels','y', 'x').hvplot.rgb(x='x', y='y', bands='channels', aspect=1, width=500)

sensor_layer + prediction_layer